In [1]:
import time
import hashlib
import json
#library to generate random number
import random
import threading

class Block:
    # Basic block init
    # Index: The position of the block in the chain starting with 0 for the genesis block
    # Timestamp: Time when the block was created
    # Transactions: Transactions included in the block
    # Previous block hash: Block hash of the previous block
    # Hash target: Block hash should be below this target
    # Nonce: Variable that miners vary to hit the hash target
    # Metadata: Any generic textual information added in the block (Optional)
    # Block hash: Hash of the current block including all the aforementioned data
    def __init__(self, index, transactions, previous_block_hash, metadata=''):
        self._index = index
        self._timestamp = time.time()
        self._transactions = transactions
        self._previous_block_hash = previous_block_hash
        self._metadata = metadata
        # self._hash_target = hash_target
        self._nonce = 0
        self._block_hash = None
        self._wait_time =0
        self._timer=None
        self.mine_block()

    def __str__(self):
        return f'\nBlock index: {self._index}\nTimestamp: {self._timestamp}\nTransactions: {self._transactions}\nPrevious Block Hash: {self._previous_block_hash}\nWait time: {self._wait_time}\nMetadata: {self._metadata}\nNonce: {self._nonce}\nBlock Hash: {self._block_hash}\n'

    #__repr__ is called on the individual elements (instead of __str__) 
    # if you try to print a list of the items or objects
    def __repr__(self):
        return self.__str__()

    @property
    def index(self):
        return self._index
    
    @property
    def block_hash(self):
        return self._block_hash

    @property
    def previous_block_hash(self):
        return self._previous_block_hash

    # @property
    # def hash_target(self):
    #     return self._hash_target

    # Serializing and utf-8 encoding relevant data, and then hashing and creating a hex representation
    def hash_block(self):
        hash_string = '-'.join([
                                str(self._index),
                                str(self._timestamp),
                                str(self._previous_block_hash),
                                str(self._metadata),
                                str(self._nonce),
                                str(self._wait_time),
                                json.dumps(self._transactions, sort_keys=True)
                                ])
        encoded_hash_string = hash_string.encode('utf-8')
        #print(f'Encoded string to be hashed for block {self._index}: {encoded_hash_string}\n\n')
        block_hash = hashlib.sha256(encoded_hash_string).hexdigest()
        return block_hash

    # Increasing nonce until block hash is below the hash target
    # Ignoring for genesis block (index 0) since that block is hard-coded
    def mine_block(self):
        #Validator requests a wait time from an enclave (a trusted function)
        wait_time=self.enclave()   
        self.create_timer()
        if(self.check_timer()):
            self._timer.start()  
            self._timer.join()          
        return True
    
    def wait_and_mine(self):
        self._block_hash = self.hash_block()
    
    def enclave(self):
        self._wait_time = random.randint(1,10)

    def create_timer(self):        
        self._timer = threading.Timer(self._wait_time, self.wait_and_mine)
    
    def check_timer(self):
        if(not self._timer):
            return False
        return True   

class Blockchain:
    # Basic blockchain init
    # Includes the chain as a list of blocks in order and pending transactions
    # Includes the current value of the hash target. It can be changed at any point to vary the difficulty
    # Also initiates a genesis block
    def __init__(self,name):
        self._name=name
        self._chain = []
        self._pending_transactions = []
        self._chain.append(self.__create_genesis_block())
        # self._hash_target = hash_target
        self._peers = []

    def __str__(self):
        return f"Chain {self._name}:\n{self._chain}\n\nPending Transactions: {self._pending_transactions}\n"

    # @property
    # def hash_target(self):
    #     return self._hash_target

    # @hash_target.setter
    # def hash_target(self, hash_target):
    #     self._hash_target = hash_target

    def add_peers(self, peers):
        self._peers.extend(peers)

    # Creating the genesis block, taking arbitrary previous block hash since there is no previous block
    # Using the famous bitcoin genesis block string here :)  
    def __create_genesis_block(self):
        genesis_block = Block(0, [], 'The Times 03/Jan/2009 Chancellor on brink of second bailout for banks', 
            None)
        return genesis_block

    # Adds block received from a peer. Only adds it if a block doesn't exist at the same height
    # It also propagates it further if this is the first time that it saw the block
    # And it doesn't send it back to the original sender
    def add_block(self, new_block, sender_node):
        if (new_block.index > self._chain[-1].index):
            self._chain.append(new_block)
            self._pending_transactions = []
            for peer in self._peers:
                if (peer != sender_node):
                    peer.add_block(new_block, self)


    # Creates a new block and appends to the chain
    # Also clears the pending transactions as they are part of the new block now
    # And sends it to all its peers since it's the originator
    def create_new_block(self):
        new_block = Block(len(self._chain), self._pending_transactions, self._chain[-1].block_hash)
        self._chain.append(new_block)
        self._pending_transactions = []
        for peer in self._peers:
            peer.add_block(new_block, self)

        return new_block

    # Adds transaction received from a peer. Only adds it if it's not already present
    # It also propagates it further if this is the first time that it saw the transaction
    # And it doesn't send it back to the original sender
    def add_transaction(self, new_transaction, sender_node):
        tx_exists = False
        for transaction in self._pending_transactions:
            if new_transaction['tx_hash'] == transaction['tx_hash']:
                tx_exists = True
        if (not tx_exists):
            self._pending_transactions.append(new_transaction)
            for peer in self._peers:
                if peer != sender_node:
                    peer.add_transaction(new_transaction, self)


    # Simple transaction with just one sender, one receiver, and one value data
    # Also calculates and stores the hash to create differentiation
    # Also sends it to all its peers since it's the originator
    def create_new_transaction(self, sender, receiver, amount, tx_metadata=''):
        new_transaction = {'timestamp': time.time(), 'sender': sender, 'receiver': receiver, 'amount': amount, 'tx_metadata': tx_metadata}
        tx_hash = hashlib.sha256(json.dumps(new_transaction, sort_keys=True).encode('utf-8')).hexdigest()
        new_transaction['tx_hash'] = tx_hash
        self._pending_transactions.append(new_transaction)
        for peer in self._peers:
            peer.add_transaction(new_transaction, self)

    # Blockchain validation function
    # Validates that previous block hash stored in each block (except genesis) is equivalent to 
    # the actual hash of the previous block
    # Also validates that each block hash is lower than the block's hash target
    def validate_blockchain(self):
        for index in range(1, len(self._chain)):
            if (self._chain[index].previous_block_hash != self._chain[index - 1].hash_block()):
                print(f'Previous block hash mismatch in block index: {index}')
                return False

            # if (int(self._chain[index].hash_block(), 16) >= int(self._chain[index].hash_target, 16)):
            #     print(f'Hash target not achieved in block index: {index}')
            #     return False
        return True


if __name__ == "__main__":
    # We'll create four blockchain nodes
    block_chain_1 = Blockchain("block_chain_1")
    block_chain_2 = Blockchain("block_chain_2")
    block_chain_3 = Blockchain("block_chain_3")
    block_chain_4 = Blockchain("block_chain_4")
    time.sleep(1)

    # We'll connect 1 to 2 and 3, and 3 to 4
    block_chain_1.add_peers([block_chain_2, block_chain_3])
    block_chain_2.add_peers([block_chain_1])
    block_chain_3.add_peers([block_chain_1, block_chain_4])
    block_chain_4.add_peers([block_chain_3])

    # Each transaction sent to a particular node will be sent to the peers who'll send it to their peers and so on..
    block_chain_1.create_new_transaction('Alice', 'Bob', 20)
    block_chain_1.create_new_transaction('Bob', 'Carol', 30)
    block_chain_2.create_new_transaction('Carol', 'Alice', 50)

    # Each mined block created on a node will be sent to the peers who'll send it to their peers and so on..
    block_chain_3.create_new_block()
    time.sleep(1)
    
    block_chain_2.create_new_transaction('Alice', 'Dave', 30)
    block_chain_3.create_new_transaction('Dave', 'Carol', 34)
    block_chain_3.create_new_transaction('Bob', 'Alice', 100)
    block_chain_4.create_new_block()
    time.sleep(1)

    block_chain_3.create_new_transaction('Alice', 'Carol', 46)
    block_chain_4.create_new_transaction('Bob', 'Dave', 90)
    
    print(block_chain_1)
    print(block_chain_2)
    print(block_chain_3)
    print(block_chain_4)
    
    validation_result = block_chain_3.validate_blockchain()
    if (validation_result):
        print('Validation successful')
    else:
        print('Validation failed')
    



Chain block_chain_1:
[
Block index: 0
Timestamp: 1631354165.2046638
Transactions: []
Previous Block Hash: The Times 03/Jan/2009 Chancellor on brink of second bailout for banks
Wait time: 6
Metadata: None
Nonce: 0
Block Hash: 62ff1c39e4600c5db1e7fbcc87f66bf410ca20e17eeadfb4370520793514de92
, 
Block index: 1
Timestamp: 1631354185.2079284
Transactions: [{'timestamp': 1631354185.2079284, 'sender': 'Alice', 'receiver': 'Bob', 'amount': 20, 'tx_metadata': '', 'tx_hash': '70d1a7da6c01b5ff31e9c9a50eda5c69dec59dfad4d8ea6a2e8cdb7ee1f18f89'}, {'timestamp': 1631354185.2079284, 'sender': 'Bob', 'receiver': 'Carol', 'amount': 30, 'tx_metadata': '', 'tx_hash': '0c1b0b266ae9dc5edee4e51b2105f5b900d3d05581e7023bd2a881969c4fbe0b'}, {'timestamp': 1631354185.2079284, 'sender': 'Carol', 'receiver': 'Alice', 'amount': 50, 'tx_metadata': '', 'tx_hash': '3e567514f2dede5f20a0ffaee1402a56b9f63a86999aa9fa9b66a9575fd24c7c'}]
Previous Block Hash: 501eb94af6863021a4f87104284981c0600105ae2a2933c346e2b12e536d665b
Wait